In [1]:
#conda install -c anaconda py-xgboost
# pip install xgboost
from xgboost import XGBClassifier

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
#import ipynb.fs.full.data_preprocessing_after_imputing as datapre

In [3]:
def best_xgboost(datatrain, datavali, parameter_space,criteria='accuracy', randomstate=0):
    '''
    if not specified, the parameters are n_estimators=100, criterion="gini", max_depth=8, min_samples_split=400, 
    min_samples_leaf=50, bootstrap=True, oob_score=False, warm_start=False, max_samples=3000, random_state=0
    '''
    from sklearn.model_selection import PredefinedSplit
    import numpy as np
    from xgboost.sklearn import XGBClassifier
    
    frames=pd.concat([datatrain, datavali])
    frames_reind=frames.set_index([pd.Index(range(len(frames)))])
    split_index=[-1]*len(datatrain)+[0]*len(datavali)
    pds = PredefinedSplit(test_fold = split_index)
    datadummy=pd.get_dummies(frames_reind)
    datatrainX=np.array(datadummy.drop('Default_ind', axis=1))
    datatrainy=datadummy['Default_ind']
    
    XGB=XGBClassifier()
    from sklearn.model_selection import GridSearchCV
    
    modelGS=GridSearchCV(XGB, parameter_space,cv=pds,scoring=criteria)
    modelGS.fit(datatrainX, datatrainy)
    
    return modelGS,datatrainX,datatrainy

In [4]:
def Cate_to_object_excp_inq(dataset):
    return(dataset.astype({'non_mtg_acc_past_due_12_months_num': 'object','non_mtg_acc_past_due_6_months_num': 'object','mortgages_past_due_6_months_num': 'object','card_inq_24_month_num': 'float64','inq_12_month_num': 'float64','card_open_36_month_num': 'object','auto_open_36_month_num': 'object','ind_acc_XYZ': 'object'}))

In [5]:
def upsample(data):
    from sklearn.utils import resample
    
    df_majority=data[data.Default_ind==0]
    df_minority=data[data.Default_ind==1]
    largenumber=len(df_majority)
    df_minority_upsampled=resample(df_minority,replace=True,n_samples=int(1.2*largenumber),random_state=123)
    df_upsampled=pd.concat([pd.DataFrame(df_majority),pd.DataFrame(df_minority_upsampled)])
    return df_upsampled

## Data Unconditional Mean

In [20]:
umean_train=pd.read_csv("Data_Set/Unconditional_mean_impute/Unconditional_mean_impute.csv")
umean_test=pd.read_csv("Data_Set/Unconditional_mean_impute/Unconditional_mean_impute_Test.csv")
umean_validation=pd.read_csv("Data_Set/Unconditional_mean_impute/Unconditional_mean_impute_Validation.csv")

In [27]:
umean_validation=umean_validation.drop(umean_validation.columns[0],axis=1)

In [29]:
#Change to categorical
cat_umean_train=Cate_to_object_excp_inq(umean_train)
cat_umean_test=Cate_to_object_excp_inq(umean_test)
cat_umean_validation=Cate_to_object_excp_inq(umean_validation)

In [30]:
# Upsample train and validation
cat_umean_up_train=upsample(cat_umean_train)
cat_umean_up_validation=upsample(cat_umean_validation)

In [31]:
datadummy=pd.get_dummies(cat_umean_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [40]:
parameter_space = {
    'colsample_bytree': [0.9],
     'gamma': [0],
     'learning_rate': [0.2],
     'max_depth': [4],
     'min_child_weight': [1],
     'n_estimators': [40],
     'nthread': [7],
     'objective': ['binary:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5]
    }

In [44]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1],
    'use_label_encoder':[False]
    }

In [45]:
clf1,x1,y1=best_xgboost(cat_umean_up_train,cat_umean_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [46]:
clf1.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0.5,
 'learning_rate': 0.3,
 'max_depth': 4,
 'min_child_weight': 0.5,
 'n_estimators': 30,
 'nthread': 4,
 'objective': 'reg:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5,
 'use_label_encoder': False}

In [47]:
y_pred=clf1.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3701,  898],
       [  87,  314]], dtype=int64)

## Listwise

In [56]:
list_train=pd.read_csv("Data_Set/Listwisedeletion/Listwise_deletion.csv")
list_test=pd.read_csv("Data_Set/Listwisedeletion/Listwise_deletion_test.csv")
list_validation=pd.read_csv("Data_Set/Listwisedeletion/Listwise_deletion_validation.csv")

In [57]:
#Change to categorical
cat_list_train=Cate_to_object_excp_inq(list_train)
cat_list_test=Cate_to_object_excp_inq(list_test)
cat_list_validation=Cate_to_object_excp_inq(list_validation)

In [71]:
# Upsample train and validation
cat_list_up_train=upsample(cat_list_train)
cat_list_up_validation=upsample(cat_list_validation)

In [72]:
datadummy=pd.get_dummies(cat_list_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [60]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1],
    'use_label_encoder':[False]
    }

In [61]:
clf2,x2,y2=best_xgboost(cat_list_up_train,cat_list_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [73]:
y_pred=clf2.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3282,  528],
       [ 105,  236]], dtype=int64)

In [76]:
clf2.best_params_

{'colsample_bytree': 1.0,
 'gamma': 0,
 'learning_rate': 0.3,
 'max_depth': 5,
 'min_child_weight': 0.5,
 'n_estimators': 50,
 'nthread': 4,
 'objective': 'reg:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5,
 'use_label_encoder': False}

## RF

In [62]:
rf_train=pd.read_csv("Data_Set/rfimpute/rf.csv")
rf_test=pd.read_csv("Data_Set/rfimpute/rf_Test.csv")
rf_validation=pd.read_csv("Data_Set/rfimpute/rf_Vali.csv")

In [66]:
#Change to categorical
cat_rf_train=Cate_to_object_excp_inq(rf_train)
cat_rf_test=Cate_to_object_excp_inq(rf_test)
cat_rf_validation=Cate_to_object_excp_inq(rf_validation)

In [67]:
# Upsample train and validation
cat_rf_up_train=upsample(cat_rf_train)
cat_rf_up_validation=upsample(cat_rf_validation)

In [74]:
datadummy=pd.get_dummies(cat_rf_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [69]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1],
    'use_label_encoder':[False]
    }

In [70]:
clf4,x4,y4=best_xgboost(cat_rf_up_train,cat_rf_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [75]:
y_pred=clf4.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3695,  904],
       [  82,  319]], dtype=int64)

In [77]:
clf4.best_params_

{'colsample_bytree': 1.0,
 'gamma': 1,
 'learning_rate': 0.3,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 40,
 'nthread': 4,
 'objective': 'reg:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5,
 'use_label_encoder': False}